# Ingest massive amounts of data to a Vector DB (Amazon OpenSearch)
**_Use of Amazon OpenSearch as a vector database for storing embeddings_**

This notebook works well with the `conda_python3` kernel on a SageMaker Notebook `ml.t3.xlarge` instance.

---
---

## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Overall Workflow](#Overall-Workflow)
1. [Step 1: Setup](#Step-1:-Setup)
1. [Step 2: Load data into OpenSearch](#Step-2:-Load-data-into-OpenSearch)
1. [Step 3: Do a similarity search for user input to documents (embeddings) in OpenSearch](#Step-3:-Do-a-similarity-search-for-for-user-input-to-documents-(embeddings)-in-OpenSearch)
1. [Conclusion](#Conclusion)

---

## Objective

This notebook illustrates how to use [`langchain`](https://python.langchain.com/en/latest/index.html), AWS Bedrock and Amazon Sagemaker Processing Job to convert large amount of data into embeddings and ingest the text data along with its embeddings into an Amazon OpenSearch index.

We use the pdf documents download from arXiv as the dataset to convert into embeddings. The Bedrock large language model (LLM) is to generate the embeddings. 

To understand the code, you might also find it useful to refer to:

- *[The langchain OpenSearch documentation](https://python.langchain.com/en/latest/ecosystem/opensearch.html)*
- *[Amazon OpenSearch service documentation](https://docs.aws.amazon.com/opensearch-service/index.html)*
- *[SageMaker Processing Job](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)*
---

## Background (Problem Description and Approach)

- **Problem statement**: 

Using LLMs for information retrieval tasks (such as question-answering) requires converting the knowledge corpus as well as user questions into vector embeddings. We want to generate these vector embeddings using an LLM hosted as a Amazon Sagemaker Endpoint and store it in a vector database of choice such as Amazon OpenSearch. For converting large amounts of data (TBs or PBs) we need a scalable system which can accomplish both converting the documents into embeddings, storing them in a vector database and provide low latency similarity search

- **Our approach**: 

1. Use Bedrock Titan FM as the LLM to generate the embeddings.

1. Place the data to be corpus of data in S3 (each document is a file stored as an object in S3).

1. Use a Python script that uses [langchain](https://python.langchain.com/en/latest/index.html) and [Opensearch-py](https://pypi.org/project/opensearch-py/) to ingest the data into OpenSearch. Run the script locally on this notebook for testing.

1. Create a Sagemaker Processing job with `instance_count` set to > 1 (usually matching the `instance_count` for the Sagemaker Endpoint). 

    Each instance of the SageMaker Processing Job runs a script that does the following:
    - Processes a subset of files from S3.
    - Uses langchain to read the files from the local filesystem and convert it into chunks.
    - Creates a langchain `OpenSearchVectorSearch` object. This is a wrapper around OpenSearch vector databases, allowing you to use it as a vectorestore for semantic search using approximate vector seach powered by licene, nmslib and faiss engines or using pailess scripting and script scoring functions for bruteforce vector search.
    - Does this using Python multiprocessing to achieve parallelization even within a single processing job instance and ensure maximum use of the Sagemaker Endpoint instance's GPU.
    > **The advantage to using langchain as a wrapper for interfacing with a vector database is that it provides a generic pattern that can be used with any LLM and any vector store. Langchain automatically uses the OpenSearch bulk ingestion API endpoint for ingesting data rather than ingesting data one record at a time. Furthermore, langchain also provides an opinionated JSON structure that includes text and metadata alongwith the embeddings itself for storing embeddings in an OpenSearch index specifically for information retrieval use-cases**.

- **Our tools**: [Amazon SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/), [langchain](https://python.langchain.com/en/latest/index.html) and [Opensearch-py](https://pypi.org/project/opensearch-py/).


---

## Overall Workflow

**Prerequisite**

The following are prerequisites that needs to be accomplised by running [this cloud formation template](./template.yaml) before running this notebook.
- A Sagemaker Endpoint for generating embeddings.
- An Amazon OpenSearch cluster for storing embeddings.
    - Opensearch cluster's access credentials (username and password) stored in AWS Secrets Mananger by following steps described [here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/managing-secrets.html).

The overall workflow for this notebook is as follows:
1. Install the required Python packages and store session information in local variables.
1. Download data from source and upload to S3.
1. Run the Python script locally to ingest a subset of data into an OpenSearch index for testing.
1. Run Sagemaker Processing Job which reads all data from S3 and runs the same Python script as above to ingest data into OpenSearch.
    - As part of this step we also create a custom container to package langchain and opensearch Python packages.
1. Do a similarity search with embeddings stored in the OpenSearch index for an input query.

---

## Step 1: Setup

Install the required packages.

In [ ]:
#Download the Bedrock dependencies.
!bash ./download-dependencies.sh

In [1]:
%pip install --no-build-isolation --force-reinstall \
    ./bedrock-python-sdk/awscli-*-py3-none-any.whl \
    ./bedrock-python-sdk/boto3-*-py3-none-any.whl \
   ./bedrock-python-sdk/botocore-*-py3-none-any.whl
%pip install --quiet langchain==0.0.249 "pypdf>=3.8,<4"
%pip install opensearch-py==2.2.0
%pip install requests
%pip install requests-aws4auth
%pip install opensearch-py

Processing ./bedrock-python-sdk/awscli-1.29.21-py3-none-any.whl
Processing ./bedrock-python-sdk/boto3-1.28.21-py3-none-any.whl
Processing ./bedrock-python-sdk/botocore-1.31.21-py3-none-any.whl
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Obtaining dependency information for s3transfer<0.7.0,>=0.6.0 from https://files.pythonhosted.org/packages/d9/17/a3b666f5ef9543cfd3c661d39d1e193abb9649d0cfbbfee3cf3b51d5af02/s3transfer-0.6.2-py3-none-any.whl.metadata
  Using cached s3transfer-0.6.2-py3-none-any.whl.metadata (1.8 kB)
  Obtaining dependency information for PyYAML<6.1,>=3.10 from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB

In [2]:
import boto3
import time
import logging, sys
from typing import List
from sagemaker.session import Session
from sagemaker.processing import ScriptProcessor, FrameworkProcessor
from sagemaker.processing import ProcessingInput
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Change the parameters if you would like to scrape a different website for data, customize chunk size etc.

In [3]:
# global constants
APP_NAME = "llm-rag-hackathon"
DATA_DIR = "data"
MAX_OS_DOCS_PER_PUT = 500
IMAGE = "load-data-opensearch-custom"
IMAGE_TAG = "latest"
CHUNK_SIZE_FOR_DOC_SPLIT = 600
CHUNK_OVERLAP_FOR_DOC_SPLIT = 20
CREATE_OS_INDEX_HINT_FILE = "_create_index_hint"
SOURCE_BUCKET = "arxiv-docs-21082003-tmp"
opensearch_index="llm-rag-hackathon"
embeddings_model_endpoint_name='amazon.titan-embed-g1-text-02'
llm_model_id="amazon.titan-tg1-xlarge"
BEDROCK_ENDPOINT_URL = "https://prod.us-west-2.frontend.bedrock.aws.dev"
#BEDROCK_REGION = "us-west-2"

logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)
logger.info(f"aws_role={aws_role}, aws_region={aws_region}, bucket={bucket}, source bucket={SOURCE_BUCKET}")

2023-09-08 18:39:59,368,1200253552,MainProcess,INFO,aws_role=arn:aws:iam::506556589049:role/LLMHackathonAppIAMRole, aws_region=us-west-2, bucket=sagemaker-us-west-2-506556589049, source bucket=arxiv-docs-21082003-tmp


### Read parameters from Cloud Formation stack

Some of the resources needed for this notebook such as the Embeddings LLM model endpoint, the Amazon OpenSearch cluster are created outside of this notebook, typically through a cloud formation template. We now read the outputs and parameters of the cloud formation stack created from that template to get the value of these parameters. 

The stack name here should match the stack name you used when creating the cloud formation stack.

In [4]:
# if used a different name while creating the cloud formation stack then change this to match the name you used
CFN_STACK_NAME = "llm-rag-hackathon"

**If you did not use a cloud formation template for creating these resources then set the names of these resources manually in the code below.**

In [5]:
stacks = boto3.client('cloudformation').list_stacks()
stack_found = CFN_STACK_NAME in [stack['StackName'] for stack in stacks['StackSummaries']]

2023-09-08 18:40:21,775,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [6]:
def get_cfn_outputs(stackname: str) -> List:
    cfn = boto3.client('cloudformation')
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

def get_cfn_parameters(stackname: str) -> List:
    cfn = boto3.client('cloudformation')
    params = {}
    for param in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Parameters']:
        params[param['ParameterKey']] = param['ParameterValue']
    return params

if stack_found is True:
    outputs = get_cfn_outputs(CFN_STACK_NAME)
    params = get_cfn_parameters(CFN_STACK_NAME)
    logger.info(f"cfn outputs={outputs}\nparams={params}")
    opensearch_domain_endpoint = f"https://{outputs['OpenSearchDomainEndpoint']}"
else:
    logger.info(f"cloud formation stack {CFN_STACK_NAME} not found, set parameters manually here")
    # REPLACE THE "placeholder" WITH ACTUAL VALUES IF YOU CREATED THESE RESOURCES WITHOUT USING A CLOUD FORMATION TEMPLATE
    opensearch_domain_endpoint = "placeholder"

2023-09-08 18:40:25,375,4289696515,MainProcess,INFO,cfn outputs={'LambdaIAMRole': 'arn:aws:iam::506556589049:role/llm-rag-hackathon-LambdaIAMRole-1J43Z0X2OS8S0', 'OpenSourceDomainArn': 'arn:aws:es:us-west-2:506556589049:domain/opensearchservi-uxkrnagpaaic', 'SageMakerIAMRole': 'arn:aws:iam::506556589049:role/LLMHackathonAppIAMRole', 'OpenSearchDomainEndpoint': 'search-opensearchservi-uxkrnagpaaic-5kqti6uvcu4b6npaxzqbyo7oom.us-west-2.es.amazonaws.com', 'SageMakerNotebookURL': 'https://console.aws.amazon.com/sagemaker/home?region=us-west-2#/notebook-instances/openNotebook/aws-llm-rag-hackathon?view=classic', 'Region': 'us-west-2', 'OpenSearchDomainName': 'opensearchservi-uxkrnagpaaic', 'LLMAppAPIEndpoint': 'https://qxtw8nfy00.execute-api.us-west-2.amazonaws.com/prod/', 'OpenSearchSecret': 'arn:aws:secretsmanager:us-west-2:506556589049:secret:OpenSearchSecret-llm-rag-hackathon-fe5x3F'}
params={'ContainerImageURI': '506556589049.dkr.ecr.us-west-2.amazonaws.com/lambda-rag-api:latest', 'Bedr

---

## Step 2: Load data into `OpenSearch`

We are now ready to create scripts which will read data from the local directory, use langchain to create embeddings and then upload the embeddings into OpenSearch.

In [7]:
"""
Create directories for storing scripts and Dockerfile
"""
!mkdir src
!mkdir data
!mkdir scripts
!mkdir container
!mkdir container/utils

mkdir: cannot create directory ‘src’: File exists
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘scripts’: File exists
mkdir: cannot create directory ‘container’: File exists
mkdir: cannot create directory ‘container/utils’: File exists


In [9]:
# create a dummy file called _create_index to provide a hint for opensearch index creation
# this is needed for Sagemaker Processing Job when there are multiple instance nodes
# all running the same code for data ingestion but only one node needs to create the index
!touch $DATA_DIR/$CREATE_OS_INDEX_HINT_FILE

!ls $DATA_DIR/$CREATE_OS_INDEX_HINT_FILE

# upload this data to S3, to be used when we run the Sagemaker Processing Job
#!aws s3 cp --recursive $DATA_DIR/ s3://$bucket/$app_name/$DOMAIN

#TODO - Ensure the Sagemaker role has access to the custom bucket. - Update IAM role. LLMAppsBlogIAMRole
!aws s3 cp $DATA_DIR/$CREATE_OS_INDEX_HINT_FILE s3://$SOURCE_BUCKET/

data/_create_index_hint
upload: data/_create_index_hint to s3://arxiv-docs-21082003-tmp/_create_index_hint


### Script to load data into OpenSearch

This script puts everything together, it divides the documents into chunks, then uses the langchain package to create embeddings and then ingests the data into OpenSearch using `OpenSearchVectorSearch`. 

To keep things simple the chunks size is set to a fixed length of 500 tokens, with an overlap of 30 tokens. The langchain `OpenSearchVectorSearch` provides a wrapper over the `opensearch-py` package. It uses the `/_bulk` API endpoint for ingesting multiple records in a single PUT request.

In [10]:
%%writefile container/load_data_into_opensearch.py

import os
import sys

# this is needed because the credentials.py and sm_helper.py
# are in /code directory of the custom container we are going 
# to create for Sagemaker Processing Job
sys.path.insert(1, '/code')

import glob
import time
import json
import os
import sys
import boto3
import logging
import argparse
import numpy as np
import multiprocessing as mp
from itertools import repeat
from functools import partial
import sagemaker, boto3, json
from typing import List, Tuple
from sagemaker.session import Session
from opensearchpy.client import OpenSearch
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from requests_aws4auth import AWS4Auth
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# global constants
MAX_OS_DOCS_PER_PUT = 500
TOTAL_INDEX_CREATION_WAIT_TIME = 60
PER_ITER_SLEEP_TIME = 5
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)


def check_if_index_exists(opensearch_domain_endpoint, index_name, region, host, http_auth) -> OpenSearch:
    
    aos_client = OpenSearch(
        hosts = [{'host': opensearch_domain_endpoint.replace("https://", ""), 'port': 443}],
        http_auth = http_auth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    exists = aos_client.indices.exists(index_name)
    logger.info(f"index_name={index_name}, exists={exists}")
    return exists

    
def process_shard(shard, embeddings_model_endpoint_name, aws_region, os_index_name, os_domain_ep, os_http_auth) -> int: 
    logger.info(f'Starting process_shard of {len(shard)} chunks.')
    st = time.time()
    embeddings = bedrock_embeddings
    
    docsearch = OpenSearchVectorSearch(index_name=os_index_name,
                                       embedding_function=embeddings,
                                       opensearch_url=os_domain_ep,
                                       http_auth=os_http_auth,
                                       is_aoss=False,
                                       use_ssl = True,
                                       verify_certs = True,
                                       timeout = 300,
                                       connection_class = RequestsHttpConnection)    
    docsearch.add_documents(documents=shard)
    et = time.time() - st
    logger.info(f'Shard completed in {et} seconds.')
    return 0


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--opensearch-cluster-domain", type=str, default=None)
    parser.add_argument("--opensearch-secretid", type=str, default=None)
    parser.add_argument("--opensearch-index-name", type=str, default=None)
    parser.add_argument("--aws-region", type=str, default="us-west-2")
    parser.add_argument("--embeddings-model-endpoint-name", type=str, default=None)
    parser.add_argument("--chunk-size-for-doc-split", type=int, default=500)
    parser.add_argument("--chunk-overlap-for-doc-split", type=int, default=30)
    parser.add_argument("--input-data-dir", type=str, default="/opt/ml/processing/input_data")
    parser.add_argument("--process-count", type=int, default=1)
    parser.add_argument("--create-index-hint-file", type=str, default="_create_index_hint")
    parser.add_argument("--bedrock-endpoint-url", type=str, default=None)
    parser.add_argument("--llm-model-id", type=str, default=None)
    args, _ = parser.parse_known_args()

    logger.info("Received arguments {}".format(args))
    # list all the files
    files = glob.glob(os.path.join(args.input_data_dir, "*.*"))
    logger.info(f"there are {len(files)} files to process in the {args.input_data_dir} folder")
    
    loader = PyPDFDirectoryLoader(path=args.input_data_dir, silent_errors=True)
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sentence, or (4) word, in that order
        chunk_size=args.chunk_size_for_doc_split,
        chunk_overlap=args.chunk_overlap_for_doc_split,
        length_function=len,
    )
    
    # Stage one: read all the docs, split them into chunks. 
    st = time.time() 
    logger.info('Loading documents ...')
    docs = loader.load()
    
    # avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
    # avg_char_count_pre = avg_doc_length(docs)
    # avg_char_count_post = avg_doc_length(docs)
    # print(f'Average length among {len(docs)} documents loaded is {avg_char_count_pre} characters.')
    # print(f'After the split we have {len(docs)} documents more than the original {len(files)}.')
    # print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')
    
    # add a custom metadata field, such as timestamp
    for doc in docs:
        doc.metadata['timestamp'] = time.time()
        doc.metadata['embeddings_model'] = args.embeddings_model_endpoint_name
    chunks = text_splitter.create_documents([doc.page_content for doc in docs], metadatas=[doc.metadata for doc in docs])
    et = time.time() - st
    logger.info(f'Time taken: {et} seconds. {len(chunks)} chunks generated') 
    
    
    db_shards = (len(chunks) // MAX_OS_DOCS_PER_PUT) + 1
    logger.info(f'Loading chunks into vector store ... using {db_shards} shards')
    st = time.time()
    shards = np.array_split(chunks, db_shards)
    
    t1 = time.time()
    
    #OpenSearch Auth
    service = 'es'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, args.aws_region, service, session_token=credentials.token)
    
    #Bedrock embeddings
    BEDROCK_ENDPOINT_URL = args.bedrock_endpoint_url
    BEDROCK_REGION = args.aws_region
  
    boto3_bedrock = boto3.client(
         service_name='bedrock',
         region_name=BEDROCK_REGION,
         endpoint_url=BEDROCK_ENDPOINT_URL
    )
    
    model_kwargs =  { 
        "maxTokenCount": 8192, 
        "stopSequences": [], 
        "temperature": 0, 
        "topP": 1 
    }


    llm = Bedrock(model_id=args.llm_model_id, client=boto3_bedrock, model_kwargs=model_kwargs)
    bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock,model_id=args.embeddings_model_endpoint_name)
    
    # first check if index exists, if it does then call the add_documents function
    # otherwise call the from_documents function which would first create the index
    # and then do a bulk add. Both add_documents and from_documents do a bulk add
    # but it is important to call from_documents first so that the index is created
    # correctly for K-NN
    logger.info(f'Checking if index exists')
    index_exists = check_if_index_exists(args.opensearch_cluster_domain,
                                         args.opensearch_index_name,
                                         args.aws_region,
                                         args.opensearch_cluster_domain,
                                         awsauth)
    
    embeddings =  bedrock_embeddings
    
    
    if index_exists is False:
        # create an index if the create index hint file exists
        path = os.path.join(args.input_data_dir, args.create_index_hint_file)
        if os.path.isfile(path) is True:
            logger.info(f"index {args.opensearch_index_name} does not exist but {path} file is present so will create index")
            
            # by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
            docsearch = OpenSearchVectorSearch.from_documents(index_name=args.opensearch_index_name,
                                                              documents=shards[0],
                                                              embedding=embeddings,
                                                              opensearch_url=args.opensearch_cluster_domain,
                                                              use_ssl = True,
                                                              verify_certs = True,  
                                                              timeout = 300,
                                                              connection_class = RequestsHttpConnection,
                                                              http_auth=awsauth)
            # we now need to start the loop below for the second shard
            shard_start_index = 1  
        else:
            logger.info(f"index {args.opensearch_index_name} does not exist and {path} file is not present, "
                        f"will wait for some other node to create the index")
            shard_start_index = 0
            # start a loop to wait for index creation by another node
            time_slept = 0
            while True:
                logger.info(f"index {args.opensearch_index_name} still does not exist, sleeping...")
                time.sleep(PER_ITER_SLEEP_TIME)
                index_exists = check_if_index_exists(args.opensearch_cluster_domain,
                                                     args.opensearch_index_name,
                                                     args.aws_region,
                                                     args.opensearch_cluster_domain,
                                                     awsauth)
                if index_exists is True:
                    logger.info(f"index {args.opensearch_index_name} now exists")
                    break
                time_slept += PER_ITER_SLEEP_TIME
                if time_slept >= TOTAL_INDEX_CREATION_WAIT_TIME:
                    logger.error(f"time_slept={time_slept} >= {TOTAL_INDEX_CREATION_WAIT_TIME}, not waiting anymore for index creation")
                    break
                
    else:
        logger.info(f"index={args.opensearch_index_name} does exist, going to call add_documents")
        shard_start_index = 0
        
    with mp.Pool(processes = args.process_count) as pool:
        results = pool.map(partial(process_shard,
                                   embeddings_model_endpoint_name=args.embeddings_model_endpoint_name,
                                   aws_region=args.aws_region,
                                   os_index_name=args.opensearch_index_name,
                                   os_domain_ep=args.opensearch_cluster_domain,
                                   os_http_auth=awsauth),
                           shards[shard_start_index:])
    
    t2 = time.time()
    logger.info(f'run time in seconds: {t2-t1:.2f}')
    logger.info("all done")

Overwriting container/load_data_into_opensearch.py


In [11]:
#Load the data in a OpenSearch index ( Local Mode )
import subprocess
def run_cmd(cmd: str) -> None:
    """
    Run a shell command. This function exists because often it is 
    cumbersome to run a shell command that takes parameters which 
    are Python variables.
    """
    MAX_OUTPUT_LEN = 20000
    logger.info(f"run_cmd, going to run cmd=\"{cmd}\"")

    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    errcode = p.returncode

    # result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    out = out.decode("utf-8") 
    err = err.decode("utf-8")
    if len(out) > MAX_OUTPUT_LEN:
        out = out[:-MAX_OUTPUT_LEN]
    if len(err) > MAX_OUTPUT_LEN:
        err = err[:-MAX_OUTPUT_LEN]
    logger.info(f"errcode={errcode}, out={out}, err={err}")

In [12]:
cmd = f"""python container/load_data_into_opensearch.py --opensearch-cluster-domain {opensearch_domain_endpoint} \
--opensearch-index-name {opensearch_index} \
--aws-region {aws_region} \
--embeddings-model-endpoint-name {embeddings_model_endpoint_name} \
--chunk-size-for-doc-split 500 \
--chunk-overlap-for-doc-split 50 \
--input-data-dir {DATA_DIR} \
--create-index-hint-file {CREATE_OS_INDEX_HINT_FILE} \
--bedrock-endpoint-url {BEDROCK_ENDPOINT_URL} \
--llm-model-id {llm_model_id} \
--process-count 2
"""
run_cmd(cmd)

2023-09-08 18:41:56,310,2701951082,MainProcess,INFO,run_cmd, going to run cmd="python container/load_data_into_opensearch.py --opensearch-cluster-domain https://search-opensearchservi-uxkrnagpaaic-5kqti6uvcu4b6npaxzqbyo7oom.us-west-2.es.amazonaws.com --opensearch-index-name llm-rag-hackathon --aws-region us-west-2 --embeddings-model-endpoint-name amazon.titan-embed-g1-text-02 --chunk-size-for-doc-split 500 --chunk-overlap-for-doc-split 50 --input-data-dir data --create-index-hint-file _create_index_hint --bedrock-endpoint-url https://prod.us-west-2.frontend.bedrock.aws.dev --llm-model-id amazon.titan-tg1-xlarge --process-count 2
"
2023-09-08 18:43:24,127,2701951082,MainProcess,INFO,errcode=1, out=sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
, err=2023-09-08 18:41:59,968,load_data_into_opensearch,MainProcess,INFO,Received argume

---

## Load the data in a `OpenSearch` index via SageMaker Processing Job (Distributed mode)

We now have a working script that is able to ingest data into an OpenSearch index. But for this to work for massive amounts of data we need to scale up the processing by running this code in a distributed fashion. We will do this using Sagemkaer Processing Job. This involves the following steps:

1. Create a custom container in which we will install the `langchain` and `opensearch-py` packges and then upload this container image to Amazon Elastic Container Registry (ECR).
2. Use the Sagemaker `ScriptProcessor` class to create a Sagemaker Processing job that will run on multiple nodes.
    - The data files available in S3 are automatically distributed across in the Sagemaker Processing Job instances by setting `s3_data_distribution_type='ShardedByS3Key'` as part of the `ProcessingInput` provided to the processing job.
    - Each node processes a subset of the files and this brings down the overall time required to ingest the data into Opensearch.
    - Each node also uses Python `multiprocessing` to internally also parallelize the file processing. Thus, **there are two levels of parallelization happening, one at the cluster level where individual nodes are distributing the work (files) amongst themselves and another at the node level where the files in a node are also split between multiple processes running on the node**.

### Create custom container

We will now create a container locally and push the container image to ECR. **The container creation process takes about 1 minute**.

1. The container include all the Python packages we need i.e. `langchain`, `opensearch-py`, `sagemaker` and `beautifulsoup4`.
1. The container also includes the `credentials.py` script for retrieving credentials from Secrets Manager and `sm_helper.py` for helping to create SageMaker endpoint classes that langchain uses.

In [13]:
%%writefile container/Dockerfile

FROM python:3.9-slim-buster

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         curl \
         unzip \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# pip leaves the install caches populated which uses a 
# significant amount of space. These optimizations save a fair 
# amount of space in the image, which reduces start up time.

# download bedrock dependencies
RUN curl https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip --output bedrock-python-sdk.zip && unzip bedrock-python-sdk.zip -d bedrock-python-sdk
RUN pip --no-cache-dir install langchain==0.0.249 opensearch-py==2.2.0 sagemaker==2.148.0 beautifulsoup4==4.12.2 "faiss-cpu>=1.7,<2" "pypdf>=3.8,<4" requests requests-aws4auth
RUN pip install --no-build-isolation --force-reinstall \
    ./bedrock-python-sdk/awscli-*-py3-none-any.whl \
    ./bedrock-python-sdk/boto3-*-py3-none-any.whl \
   ./bedrock-python-sdk/botocore-*-py3-none-any.whl

ADD utils /code/utils
COPY load_data_into_opensearch.py /code/
RUN rm -rf bedrock-python-sdk


# Set some environment variables. PYTHONUNBUFFERED keeps Python from buffering our standard
# output stream, which means that logs can be delivered to the user quickly. PYTHONDONTWRITEBYTECODE
# keeps Python from writing the .pyc files which are unnecessary in this case. We also update
# PATH so that the train and serve programs are found when the container is invoked.
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE


Overwriting container/Dockerfile


In [14]:
%%writefile scripts/build_and_push.sh

#!/usr/bin/env bash
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# The argument to this script are the path to the Dockerfile, the image name and tag and the aws-region
# in which the container is to be created. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

# override the built-in echo so that we can have a nice timestamped trace
echo () {
    builtin echo "$(date +'[%m-%d %H:%M:%S]'):" "$@"
}

if [ "$#" -eq 4 ]; then
    dlc_account_id=$(aws sts get-caller-identity | jq .Account)
    path_to_dockerfile=$1
    image=$2
    tag=$3
    region=$4
    
else
    echo "missing mandatory command line arguments, see usage..."
    echo "usage: $0 </path/to/Dockerfile> $1 <image-repo> $2 <image-tag> $3 <aws-region>"
    exit 1
fi

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"
echo the full image name would be ${fullname}

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    echo "creating ECR repository : ${fullname} "
    aws ecr create-repository --region ${region} --repository-name "${image}" > /dev/null
else
    echo "${image} repo already exists in ECR"
fi

# move to path of dockerfile
cd ${path_to_dockerfile}

# get credentials to login to ECR and, build and tag the image
# note the use of DOCKER_BUILDKIT=1, this is needed for some mount instructions in the Dockerfile
echo "going to start a docker build, image=${image}, using Dockerfile=${path_to_dockerfile}"
aws ecr get-login-password --region ${region} \
| docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com
DOCKER_BUILDKIT=1 docker build . -t ${image}  --build-arg dlc_account_id=${dlc_account_id} --build-arg region=${region}
docker tag ${image} ${fullname}
echo ${image} created

# push the image to ECR
cmd="aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com"
echo going to run \"${cmd}\" to login to ECR
${cmd}

cmd="docker push ${fullname}"
echo going to run \"${cmd}\" to push image to ecr
${cmd}
if [ $? -eq 0 ]; then
    echo "Amazon ECR URI: ${fullname}"
else
    echo "Error: Image ${fullname} build and push failed"
    exit 1
fi

echo "all done"

Overwriting scripts/build_and_push.sh


In [15]:
# Run script to build docker custom containe image and push it to ECR 
# Set region and sagemaker URI variables 
session = boto3.session.Session()
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
logger.info(f"region={aws_region}, account_id={account_id}")
!bash scripts/build_and_push.sh $(pwd)/container $IMAGE $IMAGE_TAG $aws_region

2023-09-08 18:46:13,838,383160684,MainProcess,INFO,region=us-west-2, account_id=506556589049


[09-08 18:46:15]: the full image name would be 506556589049.dkr.ecr.us-west-2.amazonaws.com/load-data-opensearch-custom:latest
[09-08 18:46:16]: load-data-opensearch-custom repo already exists in ECR
[09-08 18:46:16]: going to start a docker build, image=load-data-opensearch-custom, using Dockerfile=/home/ec2-user/SageMaker/llm-rag-hackathon/opensearch-data-ingestion/container
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.70kB                  

### Create and run the Sagemaker Processing Job

Now we will run the Sagemaker Processing Job to ingest the data into OpenSearch.

In [16]:
# setup the parameters for the job
base_job_name = f"{APP_NAME}-job"
tags = [{"Key": "data", "Value": "embeddings-for-llm-apps"}]

# use the custom container we just created
image_uri = f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{IMAGE}:{IMAGE_TAG}"

# instance type and count determined via trial and error: how much overall processing time
# and what compute cost works best for your use-case
instance_type = "ml.m5.xlarge"
instance_count = 3
logger.info(f"base_job_name={base_job_name}, tags={tags}, image_uri={image_uri}, instance_type={instance_type}, instance_count={instance_count}")

# setup the ScriptProcessor with the above parameters
processor = ScriptProcessor(base_job_name=base_job_name,
                            image_uri=image_uri,
                            role=aws_role,
                            instance_type=instance_type,
                            instance_count=instance_count,
                            command=["python3"],
                            tags=tags)

# setup input from S3, note the ShardedByS3Key, this ensures that 
# each instance gets a random and equal subset of the files in S3.
inputs = [ProcessingInput(source=f"s3://{SOURCE_BUCKET}",
                          destination='/opt/ml/processing/input_data',
                          s3_data_distribution_type='ShardedByS3Key',
                          s3_data_type='S3Prefix')]


logger.info(f"creating an opensearch index with name={opensearch_index}")
# ready to run the processing job
st = time.time()
processor.run(code="container/load_data_into_opensearch.py",
              inputs=inputs,
              outputs=[],
              arguments=["--opensearch-cluster-domain", opensearch_domain_endpoint,
                         "--opensearch-index-name", opensearch_index,
                         "--aws-region", aws_region,
                         "--embeddings-model-endpoint-name", embeddings_model_endpoint_name,
                         "--chunk-size-for-doc-split", str(CHUNK_SIZE_FOR_DOC_SPLIT),
                         "--chunk-overlap-for-doc-split", str(CHUNK_OVERLAP_FOR_DOC_SPLIT),
                         "--input-data-dir", "/opt/ml/processing/input_data",
                         "--create-index-hint-file", CREATE_OS_INDEX_HINT_FILE,
                         "--bedrock-endpoint-url", BEDROCK_ENDPOINT_URL,
                         "--llm-model-id", llm_model_id,
                         "--process-count", "2"])
time_taken = time.time() - st
logger.info(f"processing job completed, total time taken={time_taken}s")
preprocessing_job_description = processor.jobs[-1].describe()
logger.info(preprocessing_job_description)

2023-09-08 18:46:29,634,1792381935,MainProcess,INFO,base_job_name=llm-rag-hackathon-job, tags=[{'Key': 'data', 'Value': 'embeddings-for-llm-apps'}], image_uri=506556589049.dkr.ecr.us-west-2.amazonaws.com/load-data-opensearch-custom:latest, instance_type=ml.m5.xlarge, instance_count=3


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


2023-09-08 18:46:29,745,1792381935,MainProcess,INFO,creating an opensearch index with name=llm-rag-hackathon
2023-09-08 18:46:30,127,session,MainProcess,INFO,Creating processing-job with name llm-rag-hackathon-job-2023-09-08-18-46-29-746


.........................2023-09-08 18:50:40,124,load_data_into_opensearch,MainProcess,INFO,Received arguments Namespace(opensearch_cluster_domain='https://search-opensearchservi-uxkrnagpaaic-5kqti6uvcu4b6npaxzqbyo7oom.us-west-2.es.amazonaws.com', opensearch_secretid=None, opensearch_index_name='llm-rag-hackathon', aws_region='us-west-2', embeddings_model_endpoint_name='amazon.titan-embed-g1-text-02', chunk_size_for_doc_split=600, chunk_overlap_for_doc_split=20, input_data_dir='/opt/ml/processing/input_data', process_count=2, create_index_hint_file='_create_index_hint', bedrock_endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev', llm_model_id='amazon.titan-tg1-xlarge')
2023-09-08 18:50:40,124,load_data_into_opensearch,MainProcess,INFO,there are 8 files to process in the /opt/ml/processing/input_data folder
2023-09-08 18:50:40,124,load_data_into_opensearch,MainProcess,INFO,Loading documents ...
2023-09-08 18:50:40,169,load_data_into_opensearch,MainProcess,INFO,Received argume

2023-09-08 18:52:06,438,1792381935,MainProcess,INFO,processing job completed, total time taken=336.6925675868988s
2023-09-08 18:52:06,472,1792381935,MainProcess,INFO,{'ProcessingInputs': [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://arxiv-docs-21082003-tmp', 'LocalPath': '/opt/ml/processing/input_data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-506556589049/llm-rag-hackathon-job-2023-09-08-18-46-29-746/input/code/load_data_into_opensearch.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingJobName': 'llm-rag-hackathon-job-2023-09-08-18-46-29-746', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 3, 'InstanceType': 'ml.m5.xlarge', 'VolumeSizeInGB'

## Step 3: Do a similarity search for for user input to documents (embeddings) in OpenSearch

In [26]:
# Do a similarity search for user input to documents ( embeddings) in OpenSearch
import boto3
from langchain.vectorstores import OpenSearchVectorSearch
from requests_aws4auth import AWS4Auth
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from opensearchpy import RequestsHttpConnection

#OpenSearch Auth
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, aws_region, service, session_token=credentials.token)

#Bedrock embeddings
BEDROCK_ENDPOINT_URL = "https://prod.us-west-2.frontend.bedrock.aws.dev"
BEDROCK_REGION = "us-west-2"


boto3_bedrock = boto3.client(
    service_name='bedrock',
    endpoint_url=BEDROCK_ENDPOINT_URL,
    region_name=BEDROCK_REGION,
)

bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock,model_id=embeddings_model_endpoint_name)

docsearch = OpenSearchVectorSearch(index_name=opensearch_index,
                                   embedding_function=bedrock_embeddings,
                                   opensearch_url=opensearch_domain_endpoint,
                                   http_auth=awsauth,
                                   is_aoss=False,
                                   use_ssl = True,
                                   verify_certs = True,
                                   connection_class = RequestsHttpConnection) 

q = "knowledge from language models"
#docs = docsearch.similarity_search(q, k=3, search_type="script_scoring", space_type="cosinesimil")
docs = docsearch.similarity_search(q, k=3, search_type="approximate_search", space_type="cosinesimil")
for doc in docs:
    logger.info("----------")
    logger.info(f"content=\"{doc.page_content}\",\nmetadata=\"{doc.metadata}\"")

2023-09-08 19:01:51,158,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2023-09-08 19:01:51,401,base,MainProcess,INFO,POST https://search-opensearchservi-uxkrnagpaaic-5kqti6uvcu4b6npaxzqbyo7oom.us-west-2.es.amazonaws.com:443/llm-rag-hackathon/_search [status:200 request:0.051s]
2023-09-08 19:01:51,404,3401234624,MainProcess,INFO,----------
2023-09-08 19:01:51,405,3401234624,MainProcess,INFO,content="Acknowledgements
WewouldliketospeciallythankWillSutherlandforthecollaborationrep ortedinLLS
(someoftheresults ofwhich arereproduced here). Wealso wishto thankPeter Coles,
Ed Copeland, Rod Davies, Richard Frewin, Josh Frieman, Martin Hend ry, Rocky
42",
metadata="{'source': '/opt/ml/processing/input_data/astro-ph_9209003v1.The_Spectral_Index_in_the_CDM_Cosmogony.pdf', 'page': 42, 'timestamp': 1694199047.3668852, 'embeddings_model': 'amazon.titan-embed-g1-text-02'}"
2023-09-08 19:01:51,405,3401234624,MainProcess,INFO,----------
2023-09-08 19:0

---

## Cleanup

To avoid incurring future charges, delete the resources. You can do this by deleting the CloudFormation template used to create the IAM role and SageMaker notebook.

---

## Conclusion
In this notebook we were able to see how to use the BEdrock Titam model to generate embeddings and then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. We used langchain as an abstraction layer to talk to both the SageMaker Endpoint as well as OpenSearch.